# 02 – Unión con shapefile (geofencing)
Asigna una zona geográfica a cada ping, utilizando un shapefile o archivo GeoJSON de polígonos.

In [0]:
spark

In [0]:
from shapely.geometry import Point
import geopandas as gpd
import os
import pandas as pd

In [0]:
df = spark.table("sv_12_2023_limpio")

In [0]:
df.display()

In [0]:
pdf = df.select("device_id", "latitude", "longitude", "datetime").toPandas()

In [0]:
pdf = pdf.dropna(subset=["latitude", "longitude"])

Loading shapes

In [0]:
shapes_dir = "./shapes"

gdfs = []

for filename in os.listdir(shapes_dir):
    if filename.endswith(".geojson"):
      file_path = os.path.join(shapes_dir, filename)
      gdf = gpd.read_file(file_path)
      gdfs.append(gdf)

In [0]:
gdf_zonas = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
gdf_zonas = gdf_zonas.set_crs("EPSG:4326")

print(gdf_zonas.columns)
gdf_zonas.display()

In [0]:
gdf_zonas['source_layer'].value_counts()

Es posible que existan layers duplicadas por lo que haré un ranking por relevancia del layer y luego basado en ese quitarle los duplicados

In [0]:
layer_ranking = {
"gis_osm_buildings_a_free_1": 5,
"gis_osm_pois_a_free_1": 8,
"gis_osm_landuse_a_free_1": 2,
"gis_osm_traffic_a_free_1": 4,
"gis_osm_pofw_a_free_1": 7,   
"gis_osm_places_a_free_1": 6,
"gis_osm_natural_a_free_1": 1,
"gis_osm_transport_a_free_1": 3
}

In [0]:
gdf_zonas['rank'] = gdf_zonas['source_layer'].map(layer_ranking)

In [0]:
gdf_zonas_unique = gdf_zonas.sort_values('rank').drop_duplicates('osm_id', keep='first').drop(columns=['rank'])

In [0]:
print(f"Registros en DF: {len(gdf_zonas_unique)}")
print(f"Ids unicos (osm_id) en DF: {gdf_zonas_unique['osm_id'].nunique()}")

In [0]:
gdf_zonas_unique['source_layer'].value_counts()

In [0]:
#revisando si existen duplicados
duplicados = gdf_zonas_unique[gdf_zonas_unique['osm_id'].duplicated(keep=False)]

duplicados.display()

In [0]:
pdf["geometry"] = pdf.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
gdf_pings = gpd.GeoDataFrame(pdf, geometry="geometry", crs="EPSG:4326")

In [0]:
gdf_resultado = gpd.sjoin(
    gdf_zonas_unique,
    gdf_pings,
    how="inner",          # Solo los que hacen match
    predicate="contains"    # Asegura que el punto esté contenido en el polígono
)

In [0]:
gdf_resultado.display()

Guardando Geopandas como tabla en spark

In [0]:
pdf_resultado = pd.DataFrame(gdf_resultado)

In [0]:
#eliminando columna geometry para poder guardar como tabla spark para feature engineering
pdf_resultado.drop(columns=["code","geometry","population","type","index_right","latitude","longitude"], inplace=True)

In [0]:
sdf_resultado = spark.createDataFrame(pdf_resultado)

In [0]:
sdf_resultado.write.mode("overwrite").saveAsTable("sv_12_2023_ubicaciones_y_pings")

Creando tabla particionada para optimizar los queries

In [0]:
sdf_resultado.write.mode("overwrite") \
  .partitionBy("osm_id") \
  .format("delta") \
  .saveAsTable("sv_pings_georreferenciados_partitioned")